### same as the article on the website, but using averagepooling weighted with TF-IDF weights instead of max pooling

- this is roughly based upon https://github.com/fchollet/keras/blob/master/examples/imdb_fasttext.py

In [1]:
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, Dropout, Embedding, GlobalMaxPooling1D, GlobalAvgPool1D
from keras.models import Model,Sequential

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import preprocessing

from helpers import (files_helper, 
                     metrics_helper, 
                     tags_helper,
                     texts_helper)

Using TensorFlow backend.


In [2]:
SEED=np.random.randint(1,1000)
SEED

675

In [3]:
np.random.seed(SEED)

In [4]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [5]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
len(word_index),len(labels_index)

(174074, 20)

In [8]:
# make each document (sequence of word indices) be truncated to MAX_SEQUENCE_LENGTH
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [9]:
# extracting IDF weights to weight the embeddings

# does this configure snooping?
vect = TfidfVectorizer(max_features=MAX_NB_WORDS).fit(tokenized_texts)

feature_names = vect.get_feature_names()
idf = vect.idf_
idf_index = dict(zip(vect.get_feature_names(), idf))

embeddings_index = files_helper.read_glove_weighted(
    d=EMBEDDING_DIM,
    weight_index=idf_index)

((15998, 19492), (3999, 19492), (15998, 20), (3999, 20))

overall, 17826 out of 400000 embeddings were weighted


In [13]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [14]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
embedding_matrix.shape

(174075, 100)

In [16]:
model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(GlobalAvgPool1D())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(len(labels_index), activation='softmax'))

model.compile(loss ='categorical_crossentropy',
             optimizer='adam',
             metrics=['acc'])

In [17]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 2s - loss: 2.8220 - acc: 0.1311 - val_loss: 2.3607 - val_acc: 0.3373
Epoch 2/10
15998/15998 [==============================] - 2s - loss: 2.2719 - acc: 0.2618 - val_loss: 1.8766 - val_acc: 0.4344
Epoch 3/10
15998/15998 [==============================] - 2s - loss: 2.0086 - acc: 0.3215 - val_loss: 1.7099 - val_acc: 0.4611
Epoch 4/10
15998/15998 [==============================] - 2s - loss: 1.8919 - acc: 0.3567 - val_loss: 1.6152 - val_acc: 0.4861
Epoch 5/10
15998/15998 [==============================] - 2s - loss: 1.8001 - acc: 0.3792 - val_loss: 1.5353 - val_acc: 0.4966
Epoch 6/10
15998/15998 [==============================] - 2s - loss: 1.7387 - acc: 0.4028 - val_loss: 1.4981 - val_acc: 0.5034
Epoch 7/10
15998/15998 [==============================] - 2s - loss: 1.6958 - acc: 0.4171 - val_loss: 1.4538 - val_acc: 0.5271
Epoch 8/10
15998/15998 [==============================] - 2s -